In [1]:
# !python -m pip install --upgrade pip
# !python -m pip install --upgrade pyspark

# !pip install msgpack
# !pip install ipython
#!pip install pyspark==2.3.0
# !pip install pyLDAvis
# !pip install ipynb

In [1]:
import os
# os.environ["JAVA_HOME"] = "C:\Java\jdk1.8.0_172"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"
os.environ["JAVA_HOME"] = "C:/jdk1.8.0_171"
#USATO PER RISOLVEREE PROBLEMA JAVA GATEWAY ..... (ho anche installato java e inserito manualmente java home tra le variabili ambiente)

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id
import tempfile
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
import pyspark
import string
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.mllib.util import MLUtils
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
nltk.download("stopwords")
import pyLDAvis
import myLDAfunctions as mylib
from flask import Flask,jsonify,json     
from imp import reload

[nltk_data] Downloading package stopwords to C:\Users\Roberta
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Roberta
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:

reload(mylib)

[nltk_data] Downloading package stopwords to C:\Users\Roberta
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'myLDAfunctions' from 'C:\\Users\\Roberta PC\\Desktop\\Università\\LDA\\latentDirichletAllocation\\myLDAfunctions.py'>

In [3]:
spark = SparkSession.builder.appName('topicModelLDA').getOrCreate()
sc = spark.sparkContext

In [5]:
filename = "ARTICOLITV2.json"
multiline = True
df = mylib.read_our_json_dataset(filename, multiline)
clean_text, stopwords = mylib.pulizia_df_train(df, text_col = "testo_txt_it", number_of_words =100)

In [6]:
train, test = clean_text.select("final").randomSplit([0.95, 0.05])


In [8]:
df.show(5, truncate = False)

In [9]:
clean_text.show(5, truncate = False)

In [7]:
cvmodel, cvDatasetTrain, idfModel, rescaledDataTrain = mylib.tf_idf_train (train)
cvDatasetTest, rescaledDataTest = mylib.tf_idf_test (test, cvmodel, idfModel)


In [ ]:
k = 10 #numero topic
lda = LDA(k=k, seed=123, optimizer="online",featuresCol="features",maxIter = 100)
ldamodel = lda.fit(rescaledDataTrain)
# The Optimizer implements the Online variational Bayes LDA algorithm,
# which processes a subset of the corpus on each iteration,
# and updates the term-topic distribution adaptively.



In [ ]:
predictionTrain = ldamodel.transform(rescaledDataTrain)
prediction = ldamodel.transform(rescaledDataTest)

In [ ]:
vocab, doc_lengths, term_frequency, topic_term_dists, doc_topic_dists = mylib.extract_data (ldamodel, cvDatasetTrain, cvmodel, train, predictionTrain, "rawFeatures")
prep_visual = pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency )
pyLDAvis.display(prep_visual)

In [ ]:
#vocabTest, doc_lengthsTest, term_frequencyTest, topic_term_distsTest, doc_topic_distsTest = extract_data (ldamodel, cvDatasetTest, cvmodel, test, prediction, "rawFeatures")
#prep_visualTest = pyLDAvis.prepare(topic_term_distsTest, doc_topic_distsTest, doc_lengthsTest, vocabTest, term_frequencyTest )
#pyLDAvis.display(prep_visualTest)

In [13]:
# temp_path = tempfile.mkdtemp()
# ldaModelPath = temp_path + "/ldamodel"
# ldamodel.save(ldaModelPath)
# ldamodel.save("C:/lda_save/provasave")

Py4JJavaError: An error occurred while calling o374.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:96)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions$$a

In [ ]:
testo ='LeoVegas, società svedese di scommesse on line, ha presentato un reclamo alla Commissione Europea contro il decreto Dignità, sostenendo che esso "violi la legislazione dell\'Unione Europea". \
        In particolar modo il divieto assoluto di pubblicità, diretta e indiretta, legata al gioco d\'azzardo è stato adottato in violazione dell\'obbligo di notifica dell\'articolo 5 della direttiva dell\'Unione \
        Europea 2015/1535 e viola inoltre gli articoli 54 e 56 del trattato sul funzionamento dell\'Unione Europea. \
        Niklas Lindahl, Country manager di LeoVegas Italia ha dichiarato: "In questi giorni abbiamo chiesto udienza alle Commissioni Finanze e Lavoro, senza essere ricevuti, e abbiamo inutilmente proposto emendamenti al decreto Dignità"'


string_article = {"testo_txt_it":testo}
jsonRDD = sc.parallelize([json.dumps(string_article )]) 
dfnew = spark.read.json(jsonRDD) 

In [ ]:
test_clean = mylib.clean_test(dfnew, stopwords, "testo_txt_it")
cvDatasetTest, rescaledDataTestNew = mylib.tf_idf_test (test_clean, cvmodel, idfModel)




In [ ]:
test_predictions = ldamodel.transform(rescaledDataTestNew)

In [ ]:
test_predictions.show()


In [23]:
test_predictions.toPandas()

,sentence,tokenized,tokenizedNew,stemmed,final,rawFeatures,features,topicDistribution
0,leovegas societ svedese di scommesse on line...,"[leovegas, , societ, , svedese, di, scommesse,...","[leovegas, societ, svedese, di, scommesse, on,...","[leovegas, societ, svedes, di, scommess, on, l...","[leovegas, societ, svedes, scommess, reclam, c...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0012931027911979205, 0.0012955419114960543,..."


In [38]:
# vediamo = test_predictions.select("topicDistribution").toArray()

AttributeError: 'DataFrame' object has no attribute 'toArray'